In [ ]:
#  WORKOUT & DIET SUGGESTER
# Model: Hybrid (2x Classification, 1x Regression)
# Input: User's previous day calorie intake
# Output: Workout Type, Workout Duration, Foods to Avoid

# --- 1. Import Required Libraries ---
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, r2_score
import random
from IPython.display import display

# --- 2. Generate Expanded Synthetic Dataset ---
# We make the logic more continuous and realistic

num_records = 3000 # Increased record count
data = []

# Define categories
workout_types = ['Light Cardio', 'Stretching & Yoga', 'Strength Training', 'Balanced HIIT', 'Intense Cardio', 'Full Body HIIT']
food_types = ['Empty Calories', 'Processed Snacks', 'Simple Carbs', 'Excessive Fats', 'Sugary Drinks', 'Fried Foods']

for _ in range(num_records):
    # 1. Input Feature
    calories = random.randint(1500, 4500)

    # --- 2. Output Targets (Expanded Logic) ---

    # Calorie_norm: A 0-to-1 scale of calorie intake
    # 0.0 = 1500 kcal, 1.0 = 4500 kcal
    calorie_norm = (calories - 1500) / (4500 - 1500)

    # A) Workout Duration: A linear function of calories + noise
    # Base 30 mins, +40 mins potential based on calories
    base_duration = 30
    duration = base_duration + (calorie_norm * 40) + np.random.normal(0, 5)
    duration = max(15, min(90, round(duration))) # Clamp between 15-90 mins

    # B) Workout Type: Probabilities shift with calories
    if calories < 2200:
        # Low Cal: High chance of light workout
        probs = [0.4, 0.4, 0.1, 0.05, 0.025, 0.025]
    elif calories < 3000:
        # Med Cal: High chance of balanced workout
        probs = [0.1, 0.1, 0.35, 0.35, 0.05, 0.05]
    else:
        # High Cal: High chance of intense workout
        probs = [0.025, 0.025, 0.1, 0.1, 0.375, 0.375]
    workout_type = np.random.choice(workout_types, p=probs)

    # C) Foods to Avoid: Probabilities also shift
    if calories < 2200:
        probs = [0.4, 0.4, 0.1, 0.05, 0.025, 0.025]
    elif calories < 3000:
        probs = [0.1, 0.1, 0.35, 0.35, 0.05, 0.05]
    else:
        probs = [0.025, 0.025, 0.1, 0.1, 0.375, 0.375]
    foods_to_avoid = np.random.choice(food_types, p=probs)

    data.append([calories, workout_type, duration, foods_to_avoid])

# Create DataFrame
columns = ['calorie_intake', 'workout_type', 'workout_duration', 'foods_to_avoid']
df = pd.DataFrame(data, columns=columns)

print(f"✅ Expanded dataset generated with {num_records} records.")
print("--- Sample of Generated Data ---")
display(df.sample(10)) # Show 10 random samples

# --- 3. Preprocessing ---

X = df[['calorie_intake']]
y_type = df['workout_type']
y_duration = df['workout_duration']
y_food = df['foods_to_avoid']

# --- Encoding Categorical Targets ---
type_encoder = LabelEncoder()
y_type_encoded = type_encoder.fit_transform(y_type)
print("\nWorkout Type Classes:", type_encoder.classes_)

food_encoder = LabelEncoder()
y_food_encoded = food_encoder.fit_transform(y_food)
print("Foods to Avoid Classes:", food_encoder.classes_)

# --- 4. Train-Test Split ---
(X_train, X_test,
 y_type_train, y_type_test,
 y_duration_train, y_duration_test,
 y_food_train, y_food_test) = train_test_split(
    X, y_type_encoded, y_duration, y_food_encoded, test_size=0.2, random_state=42
)

print(f"\nData split: {len(X_train)} training samples, {len(X_test)} testing samples.")

# --- 5. Train the Hybrid Models ---

# Model 1: Workout Type (Logistic Regression)
print("\nTraining Model 1 (Workout Type)...")
model_type = LogisticRegression(random_state=42, max_iter=1000)
model_type.fit(X_train, y_type_train)

# Model 2: Workout Duration (Linear Regression)
print("Training Model 2 (Workout Duration)...")
model_duration = LinearRegression()
model_duration.fit(X_train, y_duration_train)

# Model 3: Foods to Avoid (Logistic Regression)
print("Training Model 3 (Foods to Avoid)...")
model_food = LogisticRegression(random_state=42, max_iter=1000)
model_food.fit(X_train, y_food_train)

print("✅ All 3 models trained successfully.")

# --- 6. (Optional) Model Evaluation ---
print("\n--- Model Performance on Test Set ---")
type_acc = accuracy_score(y_type_test, model_type.predict(X_test))
print(f"Workout Type (Classifier) Accuracy: {type_acc*100:.2f}%")
duration_r2 = r2_score(y_duration_test, model_duration.predict(X_test))
print(f"Workout Duration (Regressor) R² Score: {duration_r2:.4f}")
food_acc = accuracy_score(y_food_test, model_food.predict(X_test))
print(f"Foods to Avoid (Classifier) Accuracy: {food_acc*100:.2f}%")


# --- 7. INTERACTIVE PREDICTION LOOP ---
print("\n" + "="*40)
print("🔮 WORKOUT & DIET RECOMMENDER")
print("="*40)

try:
    while True:
        # Get user input
        try:
            cal_input_str = input("\n➡️ Enter your Previous Day's Calorie Intake (e.g., 2700) or 'exit': ")
            if cal_input_str.lower() in ['exit', 'quit']:
                print("\nGoodbye!")
                break

            cal_input = int(cal_input_str)
            if cal_input < 500 or cal_input > 8000:
                print("   Please enter a realistic calorie amount (e.g., 500-8000).")
                continue

        except ValueError:
            print("   Invalid input. Please enter a number.")
            continue

        # --- Make Predictions ---
        # Create a DataFrame for the single input
        input_df = pd.DataFrame({'calorie_intake': [cal_input]})

        # Get predictions from all 3 models
        pred_type_encoded = model_type.predict(input_df)
        pred_duration = model_duration.predict(input_df)
        pred_food_encoded = model_food.predict(input_df)

        # Decode the categorical predictions back to text
        pred_type_text = type_encoder.inverse_transform(pred_type_encoded)
        pred_food_text = food_encoder.inverse_transform(pred_food_encoded)

        # --- Display the Results ---
        print(f"\n--- 🏃‍♂️ Recommendation for {cal_input} kcal ---")
        print(f"  Suggested Workout: {pred_type_text[0]}")
        print(f"  Suggested Duration:  {pred_duration[0]:.0f} minutes")
        print(f"  Foods to Avoid:    {pred_food_text[0]}")
        print("-" * 40)

except (KeyboardInterrupt, EOFError):
    print("\n\nPrediction loop stopped.")

✅ Expanded dataset generated with 3000 records.
--- Sample of Generated Data ---


,calorie_intake,workout_type,workout_duration,foods_to_avoid
551,4486,Intense Cardio,65,Fried Foods
2742,4034,Intense Cardio,58,Excessive Fats
1997,3852,Full Body HIIT,59,Simple Carbs
1292,1637,Light Cardio,40,Processed Snacks
2316,2499,Strength Training,50,Excessive Fats
156,3422,Full Body HIIT,61,Sugary Drinks
2057,2025,Light Cardio,38,Simple Carbs
1862,2763,Balanced HIIT,41,Excessive Fats
1306,2536,Strength Training,42,Empty Calories
1917,4287,Strength Training,60,Sugary Drinks



Workout Type Classes: [np.str_('Balanced HIIT') np.str_('Full Body HIIT')
 np.str_('Intense Cardio') np.str_('Light Cardio')
 np.str_('Strength Training') np.str_('Stretching & Yoga')]
Foods to Avoid Classes: [np.str_('Empty Calories') np.str_('Excessive Fats')
 np.str_('Fried Foods') np.str_('Processed Snacks')
 np.str_('Simple Carbs') np.str_('Sugary Drinks')]

Data split: 2400 training samples, 600 testing samples.

Training Model 1 (Workout Type)...
Training Model 2 (Workout Duration)...
Training Model 3 (Foods to Avoid)...
✅ All 3 models trained successfully.

--- Model Performance on Test Set ---
Workout Type (Classifier) Accuracy: 34.83%
Workout Duration (Regressor) R² Score: 0.8360
Foods to Avoid (Classifier) Accuracy: 33.33%

🔮 WORKOUT & DIET RECOMMENDER

➡️ Enter your Previous Day's Calorie Intake (e.g., 2700) or 'exit': 4670

--- 🏃‍♂️ Recommendation for 4670 kcal ---
  Suggested Workout: Intense Cardio
  Suggested Duration:  72 minutes
  Foods to Avoid:    Fried Foods
-----